### EATD-Corpus organizer

 SDS scores are classified as 
 normal (<50)
 mild depression (50 to 59)
 moderate to marked major depression (60 to 69), 
 severe to extreme major depression (>70). The raw score can be converted to an SDS Index score by multiplying the raw score times 1.25.

In [1]:
import os
import pandas as pd

In [2]:
# read in all foldera in the dataset/EATD-Coprus folder
BASE_PATH = "datasets/EATD-Corpus"
folders = os.listdir(BASE_PATH)


In [5]:
data = []
# Loop through the folders
for folder in folders:
    # Get the path to the folder
    folder_path = os.path.join(BASE_PATH, folder)
    # Check if the folder is a directory
    if os.path.isdir(folder_path):
        # Get the path to the label.txt file
        label_path = os.path.join(folder_path, "label.txt")
        # Check if the file exists
        if os.path.exists(label_path):
            # Read the value from the file
            with open(label_path, "r") as file:
                label = file.read().strip()
            # Get the path to the new_label.txt file
            new_label_path = os.path.join(folder_path, "new_label.txt")
            # Check if the file exists
            if os.path.exists(new_label_path):
                # Read the value from the file
                with open(new_label_path, "r") as file:
                    new_label = file.read().strip()
                # Append the data to the list
                data.append([folder, label, new_label])

# Create a DataFrame from the data
df = pd.DataFrame(data, columns=["folder", "label", "new_label"])
# rename label to SDS
df.rename(columns={"label": "SDS_raw"}, inplace=True)
df.rename(columns={"new_label": "SDS_index"}, inplace=True)
# convert  SDS_raw SDS_index to float
df["SDS_raw"] = df["SDS_raw"].astype(float)
df["SDS_index"] = df["SDS_index"].astype(float)
# categorize the depression based on the SDS_index. Add new column to the dataframe
df["depression"] = pd.cut(df["SDS_index"], bins=[0, 50, 59, 69, 100], labels=["normal", "mild", "moderate", "severe"])
# add new colums: type. It can be "training" or "validation" based on the folder name (if it contains t or v )
df["type"] = df["folder"].apply(lambda x: "training" if "t" in x else "validation")
df["Participant_ID"] = df["folder"]
# if SDS is below 50 it is 0 if it is above or equal it is 1
df["PHQ8_Binary"] = df["SDS_index"].apply(lambda x: 0 if x < 50 else 1)
phgq_dict  = {'normal': 0, 'mild': 1, 'moderate': 2, 'severe': 3}
df["PHQ8_multiclass"] = df["depression"].map(phgq_dict)
df["dataset"] = df["folder"].apply(lambda x: "train" if x.startswith("t") else "dev")
df["PHQ8_Score"] = -1
"""
normal (<50)
mild depression (50 to 59)
moderate to marked major depression (60 to 69), 
severe to extreme major depression (>70).
"""
# save the dataframe to a csv file
df.to_csv("datasets/EATD-Corpus/depression_categories_EATD-Corpus.csv", index=False)